In [1]:
'''Make possible for python notebooks to import the util as module. This needs to be copied at every python notebook which wants to 
load the module'''

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [2]:
import tm_assignment_util as util
myutilObj = util.util()

importing Jupyter notebook from tm_assignment_util.ipynb
['catch', 'machine', 'accident', 'inspect', 'maintain', 'excavator', 'magnet', 'machine', 'maintenance', 'carry', 'make', 'jump', 'grappler', 'turn', 'excavator', 'engine', 'grappler', 'spin', 'pin', 'grappler', 'excavator']


In [3]:
#Let's Build the Model
# Apply preprocessing to every document in the training set
X_Toks_Trn = util.X_Cases_Trn.apply(myutilObj.my_tokenizer)
X_Toks_Tst = util.X_Cases_Tst.apply(myutilObj.my_tokenizer)

In [4]:
import logging
import gensim
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Prepare a vocabulary dictionary.
dictionary = corpora.Dictionary(X_Toks_Trn)
print(dictionary)


C:\Users\ranjanm\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-10-29 14:15:34,759 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-10-29 14:15:34,919 : INFO : built Dictionary(9136 unique tokens: ['catch', 'machine', 'accident', 'inspect', 'maintain']...) from 2894 documents (total 148208 corpus positions)


Dictionary(9136 unique tokens: ['catch', 'machine', 'accident', 'inspect', 'maintain']...)


In [5]:
# Filter off any words with document frequency less than 3, or appearing in more than 80% documents
dictionary.filter_extremes(no_below=3, no_above=0.5)
print(dictionary)

2017-10-29 14:15:34,939 : INFO : discarding 5706 tokens: [('magnet', 1), ('grappler', 1), ('supporter', 1), ('sales', 2), ('motorcycle', 1), ('ambulans', 1), ('colleagues', 1), ('reinforcement', 2), ('compaction', 2), ('gush', 2)]...
2017-10-29 14:15:34,943 : INFO : keeping 3430 tokens which were in no less than 3 and no more than 1447 (=50.0%) documents
2017-10-29 14:15:34,947 : INFO : resulting dictionary: Dictionary(3430 unique tokens: ['catch', 'machine', 'accident', 'inspect', 'maintain']...)


Dictionary(3430 unique tokens: ['catch', 'machine', 'accident', 'inspect', 'maintain']...)


In [6]:
# Use the dictionary to prepare a DTM (using TF)
dtm_train = [dictionary.doc2bow(d) for d in X_Toks_Trn]

In [7]:
# Build an LDA model for 8 topics out of the DTM
lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics = 5, id2word = dictionary, passes = 200)

2017-10-29 14:15:35,080 : INFO : using symmetric alpha at 0.2
2017-10-29 14:15:35,080 : INFO : using symmetric eta at 0.0002915451895043732
2017-10-29 14:15:35,080 : INFO : using serial LDA version on this node
2017-10-29 14:15:35,163 : INFO : running online (multi-pass) LDA training, 5 topics, 200 passes over the supplied corpus of 2894 documents, updating model once every 2000 documents, evaluating perplexity every 2894 documents, iterating 50x with a convergence threshold of 0.001000
2017-10-29 14:15:35,163 : INFO : PROGRESS: pass 0, at document #2000/2894
2017-10-29 14:15:37,975 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:15:38,006 : INFO : topic #0 (0.200): 0.014*"scaffold" + 0.010*"fall" + 0.009*"kill" + 0.009*"strike" + 0.008*"wall" + 0.007*"remove" + 0.006*"lift" + 0.006*"floor" + 0.005*"injure" + 0.005*"coworker"
2017-10-29 14:15:38,006 : INFO : topic #1 (0.200): 0.019*"fall" + 0.010*"coworker" + 0.009*"kill" + 0.008*"roof" + 0.008

2017-10-29 14:15:52,265 : INFO : topic #3 (0.200): 0.020*"ladder" + 0.010*"burn" + 0.008*"fall" + 0.008*"employer" + 0.007*"cut" + 0.007*"coworker" + 0.007*"electrical" + 0.007*"hospitalize" + 0.007*"fracture" + 0.007*"feet"
2017-10-29 14:15:52,265 : INFO : topic #4 (0.200): 0.024*"line" + 0.014*"power" + 0.014*"tower" + 0.013*"kill" + 0.013*"pole" + 0.012*"grind" + 0.010*"truck" + 0.010*"lift" + 0.009*"shock" + 0.009*"electric"
2017-10-29 14:15:52,265 : INFO : topic diff=0.398666, rho=0.474205
2017-10-29 14:15:52,265 : INFO : PROGRESS: pass 3, at document #2000/2894
2017-10-29 14:15:54,203 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:15:54,234 : INFO : topic #0 (0.200): 0.033*"scaffold" + 0.020*"fall" + 0.014*"wall" + 0.010*"fracture" + 0.009*"floor" + 0.009*"injure" + 0.008*"grind" + 0.008*"feet" + 0.008*"collapse" + 0.008*"construction"
2017-10-29 14:15:54,234 : INFO : topic #1 (0.200): 0.037*"fall" + 0.035*"roof" + 0.014*"floor" + 0.014*

2017-10-29 14:16:06,257 : INFO : topic #3 (0.200): 0.014*"ladder" + 0.011*"burn" + 0.010*"employer" + 0.008*"electrical" + 0.008*"trench" + 0.007*"cut" + 0.007*"coworker" + 0.007*"hospitalize" + 0.007*"p.m." + 0.006*"transport"
2017-10-29 14:16:06,261 : INFO : topic #4 (0.200): 0.025*"line" + 0.015*"power" + 0.014*"lift" + 0.014*"tower" + 0.014*"kill" + 0.013*"grind" + 0.012*"pole" + 0.010*"shock" + 0.010*"boom" + 0.010*"electric"
2017-10-29 14:16:06,261 : INFO : topic diff=0.221001, rho=0.366445
2017-10-29 14:16:06,265 : INFO : PROGRESS: pass 6, at document #2000/2894
2017-10-29 14:16:08,012 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:16:08,027 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.021*"fall" + 0.015*"wall" + 0.010*"fracture" + 0.010*"injure" + 0.009*"feet" + 0.009*"floor" + 0.009*"grind" + 0.008*"collapse" + 0.008*"construction"
2017-10-29 14:16:08,027 : INFO : topic #1 (0.200): 0.040*"fall" + 0.038*"roof" + 0.016*"fracture" + 0

2017-10-29 14:16:19,223 : INFO : topic #3 (0.200): 0.011*"employer" + 0.011*"burn" + 0.009*"ladder" + 0.009*"electrical" + 0.009*"trench" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"hand" + 0.007*"hospitalize" + 0.007*"service"
2017-10-29 14:16:19,223 : INFO : topic #4 (0.200): 0.025*"line" + 0.016*"lift" + 0.015*"power" + 0.014*"kill" + 0.014*"tower" + 0.013*"grind" + 0.012*"pole" + 0.012*"boom" + 0.011*"shock" + 0.010*"electric"
2017-10-29 14:16:19,223 : INFO : topic diff=0.157679, rho=0.309389
2017-10-29 14:16:19,223 : INFO : PROGRESS: pass 9, at document #2000/2894
2017-10-29 14:16:20,993 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:16:21,009 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.021*"fall" + 0.016*"wall" + 0.010*"fracture" + 0.010*"injure" + 0.010*"feet" + 0.009*"beam" + 0.009*"grind" + 0.009*"collapse" + 0.008*"floor"
2017-10-29 14:16:21,009 : INFO : topic #1 (0.200): 0.041*"fall" + 0.039*"roof" + 0.020*"ladder" + 0.017*"fract

2017-10-29 14:16:33,379 : INFO : topic #3 (0.200): 0.012*"employer" + 0.011*"burn" + 0.009*"trench" + 0.009*"electrical" + 0.007*"hand" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"service" + 0.007*"fire" + 0.007*"transport"
2017-10-29 14:16:33,379 : INFO : topic #4 (0.200): 0.025*"line" + 0.017*"lift" + 0.015*"power" + 0.014*"kill" + 0.014*"tower" + 0.013*"grind" + 0.013*"boom" + 0.012*"pole" + 0.011*"shock" + 0.010*"electric"
2017-10-29 14:16:33,379 : INFO : topic diff=0.125612, rho=0.272701
2017-10-29 14:16:33,383 : INFO : PROGRESS: pass 12, at document #2000/2894
2017-10-29 14:16:35,058 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:16:35,073 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.021*"fall" + 0.016*"wall" + 0.010*"fracture" + 0.010*"injure" + 0.010*"feet" + 0.010*"beam" + 0.009*"grind" + 0.009*"collapse" + 0.009*"concrete"
2017-10-29 14:16:35,073 : INFO : topic #1 (0.200): 0.041*"fall" + 0.039*"roof" + 0.023*"ladder" + 0.018*"fract

...

2017-10-29 14:27:29,228 : INFO : topic #3 (0.200): 0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"fire"
2017-10-29 14:27:29,232 : INFO : topic #4 (0.200): 0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"
2017-10-29 14:27:29,232 : INFO : topic diff=0.024081, rho=0.073040
2017-10-29 14:27:29,232 : INFO : PROGRESS: pass 186, at document #2000/2894
2017-10-29 14:27:30,463 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:27:30,477 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.016*"concrete" + 0.011*"injure" + 0.010*"steel" + 0.010*"collapse" + 0.010*"feet" + 0.010*"beam" + 0.010*"fracture"
2017-10-29 14:27:30,477 : INFO : topic #1 (0.200): 0.043*"fall" + 0.040*"roof" + 0.030*"ladder" + 0.021*"fract

2017-10-29 14:27:39,808 : INFO : topic #3 (0.200): 0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"fire"
2017-10-29 14:27:39,808 : INFO : topic #4 (0.200): 0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"
2017-10-29 14:27:39,808 : INFO : topic diff=0.023881, rho=0.072462
2017-10-29 14:27:39,808 : INFO : PROGRESS: pass 189, at document #2000/2894
2017-10-29 14:27:40,972 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:27:40,972 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.016*"concrete" + 0.011*"injure" + 0.010*"collapse" + 0.010*"steel" + 0.010*"feet" + 0.010*"beam" + 0.010*"fracture"
2017-10-29 14:27:40,972 : INFO : topic #1 (0.200): 0.043*"fall" + 0.040*"roof" + 0.030*"ladder" + 0.021*"fract

2017-10-29 14:27:50,180 : INFO : topic #3 (0.200): 0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"fire"
2017-10-29 14:27:50,184 : INFO : topic #4 (0.200): 0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"
2017-10-29 14:27:50,184 : INFO : topic diff=0.023686, rho=0.071898
2017-10-29 14:27:50,188 : INFO : PROGRESS: pass 192, at document #2000/2894
2017-10-29 14:27:51,486 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:27:51,498 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.016*"concrete" + 0.011*"injure" + 0.010*"collapse" + 0.010*"steel" + 0.010*"feet" + 0.010*"beam" + 0.010*"fracture"
2017-10-29 14:27:51,498 : INFO : topic #1 (0.200): 0.043*"fall" + 0.040*"roof" + 0.030*"ladder" + 0.021*"fract

2017-10-29 14:28:00,622 : INFO : topic #3 (0.200): 0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"fire"
2017-10-29 14:28:00,622 : INFO : topic #4 (0.200): 0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"
2017-10-29 14:28:00,622 : INFO : topic diff=0.023496, rho=0.071347
2017-10-29 14:28:00,626 : INFO : PROGRESS: pass 195, at document #2000/2894
2017-10-29 14:28:01,832 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:28:01,840 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.016*"concrete" + 0.011*"injure" + 0.010*"collapse" + 0.010*"steel" + 0.010*"feet" + 0.010*"beam" + 0.010*"fracture"
2017-10-29 14:28:01,840 : INFO : topic #1 (0.200): 0.043*"fall" + 0.040*"roof" + 0.030*"ladder" + 0.021*"fract

2017-10-29 14:28:10,755 : INFO : topic #3 (0.200): 0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"coworker" + 0.007*"fire"
2017-10-29 14:28:10,755 : INFO : topic #4 (0.200): 0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"
2017-10-29 14:28:10,755 : INFO : topic diff=0.023313, rho=0.070809
2017-10-29 14:28:10,755 : INFO : PROGRESS: pass 198, at document #2000/2894
2017-10-29 14:28:11,994 : INFO : merging changes from 2000 documents into a model of 2894 documents
2017-10-29 14:28:12,002 : INFO : topic #0 (0.200): 0.032*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.016*"concrete" + 0.011*"injure" + 0.010*"collapse" + 0.010*"steel" + 0.010*"feet" + 0.010*"beam" + 0.010*"fracture"
2017-10-29 14:28:12,006 : INFO : topic #1 (0.200): 0.043*"fall" + 0.040*"roof" + 0.030*"ladder" + 0.021*"fract

In [8]:
# To see the topics, with the most probable words in each topic. What topics to you see?
lda.show_topics()

[(0,
  '0.033*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.015*"concrete" + 0.011*"injure" + 0.010*"steel" + 0.010*"collapse" + 0.010*"feet" + 0.010*"fracture" + 0.010*"beam"'),
 (1,
  '0.043*"fall" + 0.041*"roof" + 0.030*"ladder" + 0.021*"fracture" + 0.019*"floor" + 0.014*"feet" + 0.014*"build" + 0.011*"hospitalize" + 0.010*"grind" + 0.009*"sustain"'),
 (2,
  '0.022*"truck" + 0.015*"kill" + 0.015*"strike" + 0.011*"burn" + 0.010*"machine" + 0.010*"pipe" + 0.009*"tank" + 0.008*"vehicle" + 0.008*"fire" + 0.007*"coworker"'),
 (3,
  '0.015*"employer" + 0.010*"burn" + 0.010*"trench" + 0.008*"service" + 0.008*"hand" + 0.008*"electrical" + 0.008*"accident" + 0.007*"p.m." + 0.007*"fire" + 0.007*"coworker"'),
 (4,
  '0.027*"line" + 0.017*"power" + 0.017*"lift" + 0.017*"boom" + 0.014*"tower" + 0.013*"kill" + 0.013*"grind" + 0.013*"pole" + 0.013*"shock" + 0.012*"electric"')]

In [9]:
# You can also request to see more words per topic
lda.show_topics(num_words=20)

[(0,
  '0.033*"scaffold" + 0.023*"fall" + 0.017*"wall" + 0.015*"concrete" + 0.011*"injure" + 0.010*"steel" + 0.010*"collapse" + 0.010*"feet" + 0.010*"fracture" + 0.010*"beam" + 0.009*"grind" + 0.009*"lift" + 0.009*"strike" + 0.008*"kill" + 0.008*"injuries" + 0.007*"hospitalize" + 0.007*"frame" + 0.007*"construction" + 0.007*"floor" + 0.007*"top"'),
 (1,
  '0.043*"fall" + 0.041*"roof" + 0.030*"ladder" + 0.021*"fracture" + 0.019*"floor" + 0.014*"feet" + 0.014*"build" + 0.011*"hospitalize" + 0.010*"grind" + 0.009*"sustain" + 0.009*"deck" + 0.009*"injure" + 0.009*"open" + 0.008*"concrete" + 0.008*"instal" + 0.008*"transport" + 0.008*"metal" + 0.007*"coworker" + 0.007*"kill" + 0.007*"step"'),
 (2,
  '0.022*"truck" + 0.015*"kill" + 0.015*"strike" + 0.011*"burn" + 0.010*"machine" + 0.010*"pipe" + 0.009*"tank" + 0.008*"vehicle" + 0.008*"fire" + 0.007*"coworker" + 0.007*"bucket" + 0.007*"water" + 0.006*"driver" + 0.006*"gas" + 0.006*"side" + 0.006*"operate" + 0.006*"move" + 0.006*"operator" + 0

In [10]:
# A similar function showing each topic with its most probable words and its topic coherence score
lda.top_topics(dtm_train)

2017-10-29 14:28:17,542 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2017-10-29 14:28:17,557 : INFO : CorpusAccumulator accumulated stats from 2000 documents


[([(0.042773026375509178, 'fall'),
   (0.04051837745361915, 'roof'),
   (0.029958592307413424, 'ladder'),
   (0.020710444188201386, 'fracture'),
   (0.01944530002880647, 'floor'),
   (0.014427477971572927, 'feet'),
   (0.014050748826057264, 'build'),
   (0.010867296195323974, 'hospitalize'),
   (0.009549287633717941, 'grind'),
   (0.0088448392006046251, 'sustain'),
   (0.0088241065912926658, 'deck'),
   (0.0088180718598536122, 'injure'),
   (0.0087837564719858117, 'open'),
   (0.0083233861551883052, 'concrete'),
   (0.008225120300184267, 'instal'),
   (0.0078111828465002768, 'transport'),
   (0.0076759329338483787, 'metal'),
   (0.0074223236601341815, 'coworker'),
   (0.0070953698970542822, 'kill'),
   (0.0070544163213575681, 'step')],
  -1.6509673112390562),
 ([(0.033283088947771149, 'scaffold'),
   (0.023106910483170574, 'fall'),
   (0.016581004597455654, 'wall'),
   (0.015483555278721209, 'concrete'),
   (0.010803027883622571, 'injure'),
   (0.010180291224656678, 'steel'),
   (0.010